起手式，導入 numpy, matplotlib

In [1]:
from PIL import Image
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('bmh')
matplotlib.rcParams['figure.figsize']=(8,5)

使用之前下載的 mnist 資料，載入訓練資料 `train_set`  和測試資料 `test_set`

In [2]:
import gzip
import pickle
with gzip.open('../Week02/mnist.pkl.gz', 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')
    
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set

之前的看圖片函數

In [3]:
from IPython.display import display
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))
# 訓練資料， X 的前 20 筆
showX(train_X[:20])

train_set 是用來訓練我們的模型用的

我們的模型是很簡單的 logistic regression 模型，用到的參數只有一個 784x10 的矩陣 W 和一個長度 10 的向量 b。

我們先用均勻隨機亂數來設定 W 和 b 。 

In [4]:
W = np.random.uniform(low=-1, high=1, size=(28*28,10))
b = np.random.uniform(low=-1, high=1, size=10)


完整的模型如下
將圖片看成是長度 784 的向量 x

計算 Wx+b， 然後再取 exp。 最後得到的十個數值。將這些數值除以他們的總和。
我們希望出來的數字會符合這張圖片是這個數字的機率。

# $softmax_i(W x + b)  = \frac {e^{W_i x + b_i}} {\sum_j e^{W_j x + b_j}}$


先拿第一筆資料試試看， x 是輸入。 y 是這張圖片對應到的數字(以這個例子來說 y=5)。

In [5]:
x = train_X[0]
y = train_y[0]
showX(x)
y

5

先計算 $e^{Wx+b} $

In [6]:
Pr = np.exp(x @ W + b)
Pr.shape

(10,)

然後 normalize，讓總和變成 1 （符合機率的意義）

In [7]:
Pr = Pr/Pr.sum()
Pr

array([  4.13597432e-03,   2.73477703e-01,   6.76290467e-02,
         2.78693575e-04,   8.96048522e-07,   5.91818687e-03,
         3.97837670e-07,   5.60042201e-01,   3.35430931e-07,
         8.85165647e-02])

由於 W 和 b 都是隨機設定的，所以上面我們算出的機率也是隨機的。

正確解是 y=5， 運氣好友可能猜中

為了要評斷我們的預測的品質，要設計一個評斷誤差的方式，我們用的方法如下（不是常見的方差，而是用熵的方式來算，好處是容易微分，效果好）



# $ error = - \log(P(Y=y^{(i)}|x^{(i)}, W,b)) $


上述的誤差評分方式，常常稱作 error 或者 loss，數學式可能有點費解。實際計算其實很簡單，就是下面的式子

In [8]:
loss = -np.log(Pr[y])
loss

5.1297251493431553

### 想辦法改進。 
我們用一種被稱作是 gradient descent 的方式來改善我們的誤差。

因為我們知道 gradient 是讓函數上升最快的方向。所以我們如果朝 gradient 的反方向走一點點（也就是下降最快的方向），那麼得到的函數值應該會小一點。

記得我們的變數是 W 和 b (裡面總共有 28*20+10 個變數)，所以我們要把 loss 對 W 和 b 裡面的每一個參數來偏微分。

還好這個偏微分是可以用手算出他的形式，而最後偏微分的式子也不會很複雜。

對 b 的偏微分如下

In [9]:
gradb = Pr.copy()
gradb[y] -= 1
print(gradb)

[  4.13597432e-03   2.73477703e-01   6.76290467e-02   2.78693575e-04
   8.96048522e-07  -9.94081813e-01   3.97837670e-07   5.60042201e-01
   3.35430931e-07   8.85165647e-02]


對 W 的偏微分也不難

In [10]:
print(Pr.shape, x.shape, W.shape)
gradW = x.reshape(784,1) @ Pr.reshape(1,10)
gradW[:, y] -= x

(10,) (784,) (784, 10)


算好 gradient 後，讓 W 和 b 分別往 gradient 反方向走一點點，得到新的 W 和 b

In [11]:
W -= 0.1 * gradW
b -= 0.1 * gradb

再一次計算 Pr 以及 loss

In [12]:
Pr = np.exp(x @ W + b)
Pr = Pr/Pr.sum()
loss = -np.log(Pr[y])
loss

0.0019717780667706259

### Q
* 看看 Pr ， 然後找出機率最大者， predict y 值
* 再跑一遍上面程序，看看誤差是否變小？
* 拿其他的測試資料來看看，我們的 W, b 學到了什麼？

我們將同樣的方式輪流對五萬筆訓練資料來做，看看情形會如何

In [13]:
W = np.random.uniform(low=-1, high=1, size=(28*28,10))
b = np.random.uniform(low=-1, high=1, size=10)
score = 0
N=50000*20
d = 0.001
learning_rate = 1e-2
for i in range(N):
    if i%50000==0:
        print(i, "%5.3f%%"%(score*100))
    x = train_X[i%50000]
    y = train_y[i%50000]
    Pr = np.exp( x @ W +b)
    Pr = Pr/Pr.sum()
    loss = -np.log(Pr[y])
    score *=(1-d)
    if Pr.argmax() == y:
        score += d
    gradb = Pr.copy()
    gradb[y] -= 1
    gradW = x.reshape(784,1) @ Pr.reshape(1,10)
    gradW[:, y] -= x
    W -= learning_rate * gradW
    b -= learning_rate * gradb
    

0 0.000%
50000 87.033%
100000 89.231%
150000 90.166%
200000 90.567%
250000 91.114%
300000 91.532%
350000 91.562%
400000 91.642%
450000 91.493%
500000 91.567%
550000 91.601%
600000 91.600%
650000 91.647%
700000 91.677%
750000 91.723%
800000 91.835%
850000 91.838%
900000 91.789%
950000 91.733%


結果發現正確率大約是 92%， 但這是對訓練資料而不是對測試資料

而且，一筆一筆的訓練資也有點慢，線性代數的特點就是能夠向量運算。如果把很多筆 $x$ 當成列向量組合成一個矩陣（然後叫做 $X$），由於矩陣乘法的原理，我們還是一樣計算 $WX+b$ ， 就可以同時得到多筆結果。

下面的函數，可以一次輸入多筆 $x$， 同時一次計算多筆 $x$ 的結果和準確率。

In [14]:
def compute_Pr(X):
    Pr = np.exp(X @ W + b)
    return Pr/Pr.sum(axis=1, keepdims=True)
def compute_accuracy(Pr, y):
    return (Pr.argmax(axis=1)==y).mean()

下面是更新過得訓練過程， 當 i%100000 時，順便計算一下 test accuracy 和 valid accuracy。

In [15]:
W = np.random.uniform(low=-1, high=1, size=(28*28,10))
b = np.random.uniform(low=-1, high=1, size=10)
score = 0
N=50000*100
batch_size = 500
learning_rate = 0.5
for i in range(0, N, batch_size):
    if i%100000==0:        
        test_score = compute_accuracy(compute_Pr(test_X), test_y)*100        
        train_score = compute_accuracy(compute_Pr(train_X), train_y)*100
        print(i, "%5.2f%%"%test_score, "%5.2f%%"%train_score)
    # 隨機選出一些訓練資料出來
    rndidx = np.random.choice(train_X.shape[0], batch_size, replace=False)
    X, y  = train_X[rndidx], train_y[rndidx]
    # 一次計算所有的 Pr
    Pr = compute_Pr(X)
    # 計算平均 gradient 
    gradb = Pr.mean(axis=0)-[(y==i).mean() for i in range(10)]
    gradW = X.T @ Pr  
    for i in range(batch_size):
        gradW[:, y[i]] -= X[i]
    gradW /= batch_size
    # 更新 W 和 b
    W -= learning_rate * gradW
    b -= learning_rate * gradb

0 12.38% 12.70%
100000 84.69% 84.26%
200000 87.28% 87.16%
300000 88.56% 88.37%
400000 89.25% 89.03%
500000 89.60% 89.71%
600000 90.14% 90.14%
700000 90.29% 90.41%
800000 90.58% 90.57%
900000 90.62% 90.76%
1000000 90.67% 91.04%
1100000 90.85% 91.27%
1200000 91.06% 91.37%
1300000 91.05% 91.47%
1400000 91.10% 91.51%
1500000 91.22% 91.68%
1600000 91.32% 91.79%
1700000 91.36% 91.80%
1800000 91.63% 91.83%
1900000 91.64% 91.80%
2000000 91.43% 92.00%
2100000 91.49% 92.05%
2200000 91.58% 92.00%
2300000 91.64% 92.15%
2400000 91.78% 92.21%
2500000 91.62% 92.20%
2600000 91.67% 92.15%
2700000 91.89% 92.34%
2800000 91.88% 92.33%
2900000 91.77% 92.31%
3000000 91.89% 92.50%
3100000 92.00% 92.43%
3200000 91.95% 92.46%
3300000 91.88% 92.43%
3400000 92.20% 92.58%
3500000 91.94% 92.62%
3600000 92.04% 92.67%
3700000 92.02% 92.60%
3800000 92.06% 92.67%
3900000 91.95% 92.71%
4000000 92.11% 92.67%
4100000 92.08% 92.73%
4200000 92.01% 92.76%
4300000 92.14% 92.79%
4400000 92.16% 92.82%
4500000 92.19% 92.80%
460

最後得到的準確率是 92%-93%

不算完美，不過畢竟這只有一個矩陣而已。

光看數據沒感覺，我們來看看前十筆測試資料跑起來的情形

可以看到前十筆只有錯一個

In [16]:
Pr = compute_Pr(test_X[:10])
pred_y =Pr.argmax(axis=1)
for i in range(10):
    print(pred_y[i], test_y[i])
    showX(test_X[i])

7 7


2 2


1 1


0 0


4 4


1 1


4 4


9 9


6 5


9 9


看看前一百筆資料中，是哪些情況算錯

In [17]:
Pr = compute_Pr(test_X[:100])
pred_y = Pr.argmax(axis=1)
for i in range(100):
    if pred_y[i] != test_y[i]:
        print(pred_y[i], test_y[i])
        showX(test_X[i])

6 5


0 4


2 3


7 6
